In [1]:
import joblib
import pandas as pd
import json

import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from eli5 import show_prediction

In [2]:
regr = joblib.load(r"D:\29 CS5228\03 Project\HDBResalePrice\outs\lgb\fold1.joblib")

In [19]:
df = pd.read_csv(r"D:\29 CS5228\03 Project\HDBResalePrice\outs\train_df_fe_all.csv")
pred = pd.read_csv(r"D:\29 CS5228\03 Project\HDBResalePrice\outs\lgb\0407_16497.04867\pred_test.csv")
with open(r"D:\29 CS5228\03 Project\HDBResalePrice\src\utils\params.json") as json_file:
    params = json.load(json_file)
nums = params['num_cols']+params['aux_cols']
cates = params['cate_cols']
cols = nums + cates
for col in nums:
    df[col] = df[col].astype(float)
for col in cates:
    df[col] = df[col].apply(lambda x: str(x).lower())
for col in cates:
    df[col] = df[col].astype('category')

In [32]:
sub_df = df.loc[list(pred['index']), cols].copy().reset_index(drop=True)
sub_df['actual'] = list(df.loc[list(pred['index']), 'resale_price'].copy())
sub_df['pred'] = list(pred['pred'].copy())
sub_df.head()

,block,flat_age,resale_year,floor_area_sqm,latitude,longitude,commercial_CBD,commercial_type_CR,commercial_type_IEBP,commercial_type_IEPB,...,street_name,storey_range,flat_model,subzone,planning_area,region,resale_quarter,resale_month,actual,pred
0,0.0,34.0,2008.0,58.0,1.303461,103.850709,1.9041,1.9041,6.8863,7.5995,...,selegie road,01 to 06,standard,bencoolen,rochor,central region,4,10,259200.0,255279.600137
1,0.0,17.0,2002.0,125.0,1.432972,103.774927,18.5094,1.3517,3.1571,15.8956,...,woodlands street 13,06 to 10,improved,woodlands west,woodlands,north region,3,8,225000.0,223921.058295
2,1.0,5.0,2002.0,106.0,1.361022,103.958601,14.2134,1.7273,2.9697,7.0890,...,tampines street 45,01 to 06,premium apartment,tampines east,tampines,east region,1,1,248850.0,247509.484989
3,1.0,12.0,2001.0,111.0,1.343720,103.689377,19.4207,5.1306,1.2286,22.9034,...,jurong west street 91,06 to 10,model a,yunnan,jurong west,west region,1,3,181800.0,187686.439078
4,0.0,31.0,2005.0,56.0,1.303709,103.850537,1.9352,1.9352,6.8694,7.5910,...,selegie road,10 to 15,standard,bencoolen,rochor,central region,1,3,166500.0,150862.991969


In [33]:
sub_df['rmse']= [
    mean_squared_error([actual], [pred], squared=False) for actual, pred in zip(sub_df['actual'], sub_df['pred'])
]
sub_df.head()

,block,flat_age,resale_year,floor_area_sqm,latitude,longitude,commercial_CBD,commercial_type_CR,commercial_type_IEBP,commercial_type_IEPB,...,storey_range,flat_model,subzone,planning_area,region,resale_quarter,resale_month,actual,pred,rmse
0,0.0,34.0,2008.0,58.0,1.303461,103.850709,1.9041,1.9041,6.8863,7.5995,...,01 to 06,standard,bencoolen,rochor,central region,4,10,259200.0,255279.600137,3920.399863
1,0.0,17.0,2002.0,125.0,1.432972,103.774927,18.5094,1.3517,3.1571,15.8956,...,06 to 10,improved,woodlands west,woodlands,north region,3,8,225000.0,223921.058295,1078.941705
2,1.0,5.0,2002.0,106.0,1.361022,103.958601,14.2134,1.7273,2.9697,7.0890,...,01 to 06,premium apartment,tampines east,tampines,east region,1,1,248850.0,247509.484989,1340.515011
3,1.0,12.0,2001.0,111.0,1.343720,103.689377,19.4207,5.1306,1.2286,22.9034,...,06 to 10,model a,yunnan,jurong west,west region,1,3,181800.0,187686.439078,5886.439078
4,0.0,31.0,2005.0,56.0,1.303709,103.850537,1.9352,1.9352,6.8694,7.5910,...,10 to 15,standard,bencoolen,rochor,central region,1,3,166500.0,150862.991969,15637.008031


In [35]:
sub_df['weight_rmse']=sub_df['rmse']/sub_df['actual']
sub_df.head()

,block,flat_age,resale_year,floor_area_sqm,latitude,longitude,commercial_CBD,commercial_type_CR,commercial_type_IEBP,commercial_type_IEPB,...,flat_model,subzone,planning_area,region,resale_quarter,resale_month,actual,pred,rmse,weight_rmse
0,0.0,34.0,2008.0,58.0,1.303461,103.850709,1.9041,1.9041,6.8863,7.5995,...,standard,bencoolen,rochor,central region,4,10,259200.0,255279.600137,3920.399863,0.015125
1,0.0,17.0,2002.0,125.0,1.432972,103.774927,18.5094,1.3517,3.1571,15.8956,...,improved,woodlands west,woodlands,north region,3,8,225000.0,223921.058295,1078.941705,0.004795
2,1.0,5.0,2002.0,106.0,1.361022,103.958601,14.2134,1.7273,2.9697,7.0890,...,premium apartment,tampines east,tampines,east region,1,1,248850.0,247509.484989,1340.515011,0.005387
3,1.0,12.0,2001.0,111.0,1.343720,103.689377,19.4207,5.1306,1.2286,22.9034,...,model a,yunnan,jurong west,west region,1,3,181800.0,187686.439078,5886.439078,0.032379
4,0.0,31.0,2005.0,56.0,1.303709,103.850537,1.9352,1.9352,6.8694,7.5910,...,standard,bencoolen,rochor,central region,1,3,166500.0,150862.991969,15637.008031,0.093916


In [37]:
sub_df[sub_df['weight_rmse']>0.5]

,block,flat_age,resale_year,floor_area_sqm,latitude,longitude,commercial_CBD,commercial_type_CR,commercial_type_IEBP,commercial_type_IEPB,...,flat_model,subzone,planning_area,region,resale_quarter,resale_month,actual,pred,rmse,weight_rmse
778,0.0,40.0,2020.0,67.0,1.374431,103.850157,9.7598,4.3105,2.4921,5.4206,...,new generation,cheng san,ang mo kio,north-east region,3,7,156600.0,272150.805360,115550.805360,0.737872
1131,1.0,29.0,2002.0,64.0,1.284206,103.831063,2.6241,2.6241,4.8393,10.6492,...,standard,tiong bahru,bukit merah,central region,3,7,83700.0,132486.000201,48786.000201,0.582867
2877,1.0,18.0,2000.0,145.0,1.352908,103.888972,8.2915,6.2492,2.8949,0.7352,...,adjoined flat,lorong ah soo,hougang,north-east region,2,6,163800.0,318268.420950,154468.420950,0.943031
6017,0.0,29.0,2005.0,60.0,1.272333,103.822210,3.9350,3.9350,4.3115,12.2900,...,improved,telok blangah rise,bukit merah,central region,3,9,81000.0,122452.173488,41452.173488,0.511755
7472,1.0,20.0,2005.0,64.0,1.351145,103.726473,15.9338,2.0969,3.4769,18.7485,...,simplified,hong kah,jurong west,west region,1,3,85500.0,143426.891398,57926.891398,0.677508
9266,0.0,25.0,2000.0,153.0,1.303178,103.911807,6.6237,6.6237,6.9103,6.1035,...,adjoined flat,marine parade,marine parade,central region,3,7,180000.0,417528.256119,237528.256119,1.319601
12692,0.0,28.0,2003.0,141.0,1.304563,103.917229,7.2456,6.2357,6.3166,6.1718,...,adjoined flat,marine parade,marine parade,central region,3,7,198000.0,303744.016424,105744.016424,0.534061
13693,2.0,27.0,2000.0,121.0,1.298952,103.798542,6.3696,6.3696,1.0677,12.4352,...,adjoined flat,tanglin halt,queenstown,central region,2,5,130500.0,242430.759143,111930.759143,0.857707
27906,1.0,32.0,2001.0,120.0,1.325410,103.885651,5.5166,5.5166,5.8506,3.5015,...,adjoined flat,macpherson,geylang,central region,2,5,108000.0,184767.693547,76767.693547,0.710812
32353,1.0,26.0,2003.0,154.0,1.319880,103.741754,13.0662,1.7729,1.0918,17.4969,...,adjoined flat,teban gardens,jurong east,west region,1,2,91800.0,223598.409136,131798.409136,1.435713


In [42]:
for col in cates:
    sub_df[col] = sub_df[col].astype('category')
    
show_prediction(regr, np.array(sub_df.loc[778, cols]), feature_names=cols, feature_filter=nums, show_feature_values=True)

ValueError: could not convert string to float: '3 room'